In [7]:
# import packages
import requests, json
import pandas as pd, numpy as np
import time, os
from bs4 import BeautifulSoup
import re
from datetime import datetime



In [183]:
def prepros(html) :
    d=json.loads(html)
    
    soup = BeautifulSoup(d['result_list_box_html'],'lxml')
    
    # this selects the main part of the html
    tabel = soup.find('div', attrs={'class':'results component--default'})
    
    # Jobindex contain both own postings and postings from other
    # the two are treated differently in the data
    # Jobindex' imported postings
    others = re.compile('r[\d]+')
    tabel1 = tabel.findAll('div', attrs={'data-beacon-tid':others})
#    print(tabel1)

    # Jobindex' own postings
    own = re.compile('h[\d]+')
    tabel2 = tabel.findAll('div', attrs={'data-beacon-tid':own})
    #print(tabel2)
    
    return tabel, tabel1, tabel2

def job_descr_own(tabel2) : # den er ikke færdig: tag <ul> og saml elementer, der hører sammen
    desc_own = []

    # tag ul skal med 
    for l in tabel2 :
        g = l.findAll('p')
        for m in g :
            desc_own.append(m.text)
    return desc_own

def job_descr_oth(tabel1) : # færdig og virker
    desc_oth = []
    p=re.compile('"')
    lineshift = re.compile('\n')
    besk = []
    for t_ in tabel1 :
        besk.append(lineshift.sub("", t_.text))
    for t in besk :
        s = t.split(sep='    ')
        if len(s) == 1 :
            desc_oth.append("")
        else :
            desc_oth.append(t.split(sep='    ')[1])
    return desc_oth
    
def job_title_oth (tabel) : # færdig og virker
    # udled jobs andre
    jobs_oth = []
    j = tabel.findAll('strong') 
    for l in j :
        jobs_oth.append(l.text)
    return jobs_oth


def firm_place(tabel) : # tjek den igen, der kommer lidt for mange elementer ud
    firm=[]
    city=[]
    for tag in tabel :
        firm_city=tag.findAll('b')
        if len(firm_city) == 0 :
            firm.append("Ukendt")
            city.append("Uoplyst")
        elif len(firm_city) == 1 :
            firm.append(firm_city[0].text)
            city.append(" ")
        else :
            firm.append(firm_city[0].text)
            city.append(firm_city[1].text)
    return firm, city


def dates(tabel) : # finds only dates for others; needs also dates for own
    indented_d=[]
    for tag in tabel :
        dato_site=tag.findAll('time')

        monthval={'januar':'01','februar':'02', 'marts':'03', 'april':'04', 'maj':'05', 'juni':'06', 'juli':'07','august':'08','september':'09','oktober':'10','november':'11','december':'12'}

        for i in dato_site:
            t = i.text.split()
            try :
                datotal=(t[0][:-1]+ monthval.get(t[1])+ t[2])
                indented_d.append(time.mktime(datetime.strptime(str(datotal), '%d%m%Y').timetuple()))
            except :
                indented_d.append(time.time()) # vi bør videreføre sidst kendte værdi også på tværs af sider

    return indented_d



#hyper = []
#x1 = tabel.findAll('a', attrs={'href':True}) 
#for x in x1 :
#    hyper.append(x.)
#    
#print(x1)



In [181]:
# select small subsample
import random, time

ca_samplesize = 1000
base_path = r"C:\Notebooks/jobindex.txt"
sample_path = r"C:\Notebooks\group19\Eksamensprojekt/jobindex_sample.txt"
f = open(base_path,'r')
s = open(sample_path, 'w')
linienr = 0
t0 = time.time()
for line in f :
    ran = random.uniform(0, 1) 
    if ran < ca_samplesize/17000 :
        linienr += 1
        s.write(line)
f.close()
t1 = time.time()
print("Start:", t0, ". Slut:", t1, ". Linier hentet:", linienr)
print("Kørseltid:", round(t1-t0,2), "sekunder")




Start: 1535289323.456111 . Slut: 1535289338.7372632 . Linier hentet: 989
Kørseltid: 15.28 sekunder


In [185]:

def process_data() :
    t0 = time.time()
    base_path = r"C:\Notebooks\group19\Eksamensprojekt/jobindex_sample.txt" # file with scraped jobindex data

    f = open(base_path, 'r') # open the file for reading
    count = 0
    # loop through the file one line at a time
    
    
    for html in f :
        count += 1
#        print(count, len(html))
        if len(html) == 1 :
            continue
        # here the various processing functions will be called
        tabel, tabel1, tabel2= prepros(html)
#        print(len(tabel))

        desc_own = job_descr_own(tabel2)
#        print("desc_own", len(desc_own), desc_own)

        desc_oth = job_descr_oth(tabel1)
#        print('desc_oth', len(desc_oth), desc_oth)
        
        jobs_oth = job_title_oth(tabel)
#        print('jobs_oth', len(jobs_oth), jobs_oth)
        
        firm_oth, city_oth = firm_place(tabel1)

        post_dates_oth = dates(tabel1)
#        print("dates", len(post_dates_oth))

        post_dates_own = dates(tabel1)
#        print("dates", len(post_dates_own), post_dates_own)

    # der skal laves en zip-funktion, så de kan blive sat rigtig sammen
#    return desc_own, desc_oth, jobs_oth, firm, city, post_dates
        output_oth = list(zip(post_dates_oth, jobs_oth, desc_oth, firm_oth, city_oth))
        output_own = list(zip(post_dates_own, desc_own))
    f.close()
    t1 = time.time()
    print("Procestid er", round(t1-t0, 2), "sekunder")
    return output_oth, output_own

output_oth, output_own = process_data()




# gem data
other = pd.DataFrame(output_oth)
other.to_pickle('others.pkl')

own = pd.DataFrame(output_own)
own.to_pickle('own.pkl')


#print(len(desc_own))
#print(len(desc_oth))
#print(len(jobs_oth))
#print(len(firm))
#print(len(city))
#print(len(post_dates))

Procestid er 56.91 sekunder


""


In [189]:

# læs data 
ny = pd.read_pickle('others.pkl')

ny.tail()



,0,1,2,3,4
15,1.211148e+09,Produktionsassistent til fødevarevirksomhed - ...,Som produktionsassistent er du med til at styr...,Ukendt,Uoplyst
16,1.211148e+09,Administrationsmedarbejder søges til internati...,For vores kunde en international luftfragtvirk...,Kastrup,
17,1.211148e+09,Ordrebehandler til sommervikariat,Du vil kunne forvente at skulle løse en række ...,København,
18,1.211148e+09,Produktionsassistent til fødevarevirksomhed - ...,Som produktionsassistent er du med til at styr...,Ukendt,Uoplyst
19,1.211148e+09,Sommerblæksprutte til kontorarbejde,Kontorassistent til 2 måneders vikariat i somm...,København,
